In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [1]:
from os.path import join,isfile,isdir

modelPath='/content/drive/MyDrive/kobert/Model'
inputPath='/content/drive/MyDrive/kobert/Data-input'

dPath=join(inputPath, "Test.tsv")


In [3]:
%%bash
echo mkdir -p {modelPath}
echo mkdir -p {inputPath}

mkdir -p {modelPath}
mkdir -p {inputPath}


In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-jvym4qzn
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-jvym4qzn
     |████████████████████████████████| 129 kB 15.0 MB/s 
     |████████████████████████████████| 344 kB 73.3 MB/s 
     |████████████████████████████████| 54.7 MB 39 kB/s 
     |████████████████████████████████| 4.5 MB 25.1 MB/s 
     |████████████████████████████████| 1.2 MB 70.5 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.2 MB/s eta 0:00:40tcmalloc: large alloc 1147494400 bytes == 0x394e4000 @  0x7f63d5216615 0x5d6f4c 0x51edd1 0x51ef5b 0x4f750a 0x4997a2 0x4fd8b5 0x4997c7 0x4fd8b5 0x49abe4 0x4f5fe9 0x55e146 0x4f5fe9 0x55e146 0x4f5fe9 0x55e146 0x5d8868 0x5da092 0x587116 0x5d8d8c 0x55dc1e 0x55cd91 0x5d8941 0x49abe4 0x55cd91 0x5d8941 0x4990ca 0x5d8868 0x4997a2 0x4fd8b5 

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pickle


In [ ]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW # 인공지능 모델의 초기값 지정 함수를 아담으로 지정한다.
from transformers.optimization import get_cosine_schedule_with_warmup


In [ ]:
device = torch.device("cuda:0")

NameError: ignored

In [ ]:
# 함수 정의

def calc_accuracy(X,Y):
  max_vals, max_indices = torch.max(X, 1)
  train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
  return train_acc

def get_1d(l):
  ll=[]
  for i in l:
    for j in i:
      ll.append(j)
  return ll

def calc_accuracy2(x, y):
  xx = get_1d(x)
  yy = get_1d(y)
  xl=len(xx)
  cnt=0
  if xl==len(yy):
    for i in range(xl):
      if xx[i] == yy[i]:
        cnt+=1
    return cnt/xl

def isEqual(X, Y):
  max_vals, max_indices = torch.max(X, 1)
  if max_indices.cpu().numpy()[0] == Y:
    return 1
  else:
    return 0

In [ ]:
def corderReversor(corder):
  k=corder.keys()
  return { corder[i]:i for i in k}



In [ ]:
#data loading

import pandas as pd

df = pd.read_csv(dPath, sep="\t")

In [ ]:
# Encoder, Decoder
keyword_decoder={k[0]:k[1] for k in enumerate(df.keyword.unique() )}
keyword_encoder=corderReversor(keyword_decoder)

In [ ]:
keywordlist=df.keyword.tolist()

df['label']=pd.Series([keyword_encoder[i] for i in keywordlist])

data_list = []
for q, label in zip(df['sentence'], df['label']):
  data = []
  data.append(q)
  data.append(str(label))

  data_list.append(data)



In [ ]:
?zip

In [ ]:
data_list

[['나는 감자를 먹는다.', '0'],
 ['그녀는 농구를 한다.', '1'],
 ['재욱이는 밥을 먹고 싶다.', '2'],
 ['그는 집을 좋아한다.', '3'],
 ['친구가 옷을 입는다.', '4'],
 ['이사님은 차를 타신다.', '5'],
 ['차대리님은 공을 찬다.', '6'],
 ['사원님은 하키를 한다.', '7'],
 ['걔네는 사진을 찍어.', '8'],
 ['우리 농구를 하자.', '1'],
 ['나는 음악을 듣는다.', '9'],
 ['음식에 소스를 뿌린다.', '10'],
 ['나는 사랑을 느낀다.', '11'],
 ['도준이는 순양을 산다.', '12'],
 ['나는 가족을 사랑한다.', '13'],
 ['미국은 총기를 허가했다.', '14'],
 ['세상은 정재욱을 원한다.', '15'],
 ['빙수에 초코를 뿌리자.', '16'],
 ['철수는 컴퓨터를 한다.', '17'],
 ['영희는 비행기를 타고 있다.', '18'],
 ['학교는 애들을 올바른 길로 이끈다.', '19'],
 ['아르헨티나가 월드컵을 들어올렸다.', '20'],
 ['메시가 우승을 하였다.', '21'],
 ['호날두는 한국을 도왔다.', '22'],
 ['나는 강아지를 좋아한다.', '23'],
 ['그녀는 고양이를 산책시켰다.', '24'],
 ['우리는 난민을 도와야한다.', '25'],
 ['나는 파라미터를 조정해야한다.', '26'],
 ['나는 프로젝트를 끝내고 싶다.', '27']]

# 데이터 전처리

In [ ]:
dataset_train=data_list
dataset_test=data_list.copy()

num_classes = df.keyword.unique().shape[0]
print(num_classes)

28


In [ ]:
# model 준비
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [ ]:
# Model Training

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=28,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
# 인코더(토큰화, 패딩)
class BERTDataset(Dataset):
  def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
    transform = nlp.data.BERTSentenceTransform(
        bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
    
    self.sentences = [transform([i[sent_idx]]) for i in dataset]
    self.labels = [np.int32(i[label_idx]) for i in dataset]

  def __getitem__(self, i):
    return (self.sentences[i] + (self.labels[i], ))

  def __len__(self):
    return (len(self.labels))

In [ ]:
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 50
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
data_train[0]

(array([   2, 1375,  784, 7158, 2010, 5760, 5782,  517,   54,    3,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(10, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 0)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
#BERT 모델 불러오기
# model = BERTClassifier(bertmodel,num_classes=2,  dr_rate=0.5).to(device)
model = BERTClassifier(bertmodel,num_classes=num_classes,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-22-480b6a139979>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/1 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 3.4717178344726562 train acc 0.034482758620689655
epoch 1 train acc 0.034482758620689655


<ipython-input-22-480b6a139979>:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/1 [00:00<?, ?it/s]

...

epoch 49 test acc 0.8620689655172413


  0%|          | 0/1 [00:00<?, ?it/s]

epoch 50 batch id 1 loss 2.556562662124634 train acc 0.5517241379310345
epoch 50 train acc 0.5517241379310345


  0%|          | 0/1 [00:00<?, ?it/s]

epoch 50 test acc 0.8620689655172413


In [ ]:
# Model Save
torch.save(model.state_dict(),join(modelPath,"bert_test" ))

In [ ]:
# Model Load
model = BERTClassifier(bertmodel,  dr_rate=0.5, num_classes=num_classes ).to(device)
model.load_state_dict(torch.load(join(modelPath,"bert_test" )) )

<All keys matched successfully>

In [ ]:
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

i=0
# cnt=0
test_acc=0
labels=[]
results=[]
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
      i+=1
      token_ids = token_ids.long().to(device)
      segment_ids = segment_ids.long().to(device)
      valid_length= valid_length
      label = label.long().to(device)
      out = model(token_ids, valid_length, segment_ids)
      acc=calc_accuracy(out, label)
      results.append(torch.max(out, 1)[1].cpu().numpy().tolist())
      labels.append(label.cpu().numpy().tolist())
      # print(calc_accuracy(out, label))

      # test_acc += calc_accuracy(out, label)
# lable1=label.cpu().numpy()
result=torch.max(out, 1)[1].cpu().numpy()
print('-----------------')

print("test_acc %: ",calc_accuracy2(labels,results))
print("label  : ", labels)
print('result : ', results )



<ipython-input-25-024e7792074f>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/1 [00:00<?, ?it/s]

-----------------
test_acc %:  0.6551724137931034
label  :  [[0, 1, 2, 3, 4, 5, 6, 7, 8, 1, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]]
result :  [[0, 1, 2, 21, 4, 5, 1, 7, 8, 1, 25, 10, 11, 12, 11, 14, 15, 16, 12, 12, 11, 20, 20, 22, 23, 25, 25, 11, 27]]


In [ ]:

#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)


using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
def predict(predict_sentences):
  # predict_sentence="의사"
  dataset_another = [[predict_sentence, '0'] for predict_sentence in predict_sentences]
  # dataset_another = [data]

  another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
  test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

  model.eval()

  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)

    valid_length= valid_length
    label = label.long().to(device)

    out = model(token_ids, valid_length, segment_ids)


    test_eval=[]
    for i in out:
      logits=i
      logits = logits.detach().cpu().numpy()
      test_eval.append(np.argmax(logits))
  return test_eval



In [ ]:
[keyword_decoder[i] for i in predict([
    '재욱이는 밥을 먹고 싶다.',
    '빙수에 초코를 뿌리자.',
    '학교는 애들을 올바른 길로 이끈다.',
    '도준이는 순양을 산다.'
])  ]

['밥', '초코', '애들', '순양']